In [10]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [11]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [12]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['compositional-abstractions']
coll = db['two-towers']

# which iteration name should we use?
iterationName = 'testing'

## Sanity Checks

In [6]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

# query = coll.find({"$and":[
# #                         {'workerId':{'$exists':True}},
# #                         {'condition':{'$ne':'practice'}},
# #                         {'eventType':'trial_end'},
#                         {"$or":[{'iterationName':'testing'}]}]
#                      })

#df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]

query = coll.find()

df_trial_end_full = pd.DataFrame(list(query))

#assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [16]:
## How many records?
coll.estimated_document_count()

0

In [7]:
df_trial_end_full.columns

Index(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3',
       '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40',
       '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51',
       '52', '53', '54', '55', '56', '57', '6', '7', '8', '9', '_id',
       'assignmentId', 'block', 'blockNum', 'color', 'comments', 'confused',
       'content', 'discreteWorld', 'eventType', 'gameID', 'gameid', 'height',
       'isHuman', 'iterationName', 'leftTarget', 'nativeEnglish', 'repNum',
       'rightTarget', 'role', 'score', 'strategy', 'targetMap', 'time',
       'timeElapsed', 'timeElapsedInTrial', 'timeElapsedInTurn', 'totalLength',
       'trialNum', 'trialStartTime', 'turnNum', 'turnStartTime',
       'turnTimeElapsed', 'width', 'workerId', 'x', 'y'],
      dtype='object')

In [8]:
df_trial_end_full[['_id', 'iterationName', 'gameid', 'time', 'workerId', 'assignmentId',\
       'leftTarget', 'rightTarget', 'trialNum', 'turnNum', 'repNum', 'content',\
       'eventType', 'blockNum','x', 'y', 'width', 'height', 'color','discreteWorld']].tail(5)

,_id,iterationName,gameid,time,workerId,assignmentId,leftTarget,rightTarget,trialNum,turnNum,repNum,content,eventType,blockNum,x,y,width,height,color,discreteWorld
336,5ee5121bacd2500c287c0818,testing,5437-8bbc3650-65ea-432f-85d8-74683b8e37fb,1.592071e+12,A4SSYO0HDVD4E,3JMSRU9HQMURRU88JI67UQFXEU5VE4,L,Pi,5,1,3,NaN,block,5.0,10.0,0.0,1.0,2.0,"[179, 47, 10, 155]","[[False, False, False, False, False, False, Fa..."
337,5ee5121cacd2500c287c0819,testing,5437-8bbc3650-65ea-432f-85d8-74683b8e37fb,1.592071e+12,A4SSYO0HDVD4E,3JMSRU9HQMURRU88JI67UQFXEU5VE4,L,Pi,5,1,3,NaN,block,6.0,7.0,2.0,2.0,1.0,"[10, 47, 179, 155]","[[False, False, False, False, False, False, Fa..."
338,5ee5121dacd2500c287c081a,testing,5437-8bbc3650-65ea-432f-85d8-74683b8e37fb,1.592071e+12,A4SSYO0HDVD4E,3JMSRU9HQMURRU88JI67UQFXEU5VE4,L,Pi,5,1,3,NaN,block,7.0,9.0,2.0,2.0,1.0,"[10, 47, 179, 155]","[[False, False, False, False, False, False, Fa..."
339,5ee5121dacd2500c287c081b,testing,5437-8bbc3650-65ea-432f-85d8-74683b8e37fb,1.592071e+12,,,L,Pi,5,0,3,NaN,endTrial,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,5ee513daacd2500c287c081c,testing,5437-8bbc3650-65ea-432f-85d8-74683b8e37fb,1.592071e+12,A4SSYO0HDVD4E,3JMSRU9HQMURRU88JI67UQFXEU5VE4,L,Pi,NaN,0,3,NaN,exitSurvey,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Trial Level Data

In [196]:
# Assuming that if trial 23 saves, then 0-22 have also saved 
# get ids of people with trial 23 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'Exp2Pilot3'},
                                {'iterationName':'Exp2Pilot3_batch2'}]},
                        #{'iterationName': iterationName}, #use this if one iteration name
                        {'trialNum': numTrials-1}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [197]:
# Filter for full datasets
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        #{'iterationName': iterationName}, #use this if one iteration name
                        {"$or":[{'iterationName':'Exp2Pilot3'},
                                {'iterationName':'Exp2Pilot3_batch2'}]}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))


# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]

# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[[
    'gameID','trialNum','phase','condition','eventType','targetName','repetition','targetID', #trial identifiers
    'nullScore','F1Score','normedScore','rawScoreDiscrete','nullScoreDiscrete','normedScoreDiscrete','scoreGapDiscrete', #scoring
    'numBlocks','nPracticeAttempts','blockColor','blockColorID','blockFell','doNothingRepeats',#misc. trial info
    'score','currBonus','timeBonus', #bonusing
    'timeAbsolute','timeRelative','buildTime','buildStartTime','buildFinishTime','timeToBuild', #timing 
    'discreteWorld','allVertices', #world reconstruction
    'browser','browserVersion','os','devMode', #developer info
    #below here should be the same for every trial in a dataset
    'iterationName',
    'numTargets', 'prePostSetSize','numRepetitions', #pre-post info
    'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh','timeThresholdYellow','timeThresholdRed', #bonus info
    ]]

#Fix error in data-saving- normedScoreDiscrete saved as rawScoreDiscrete
df_trial_end_reduced_filtered['normedScoreDiscrete'] = df_trial_end_reduced_filtered['rawScoreDiscrete']
df_trial_end_reduced_filtered.drop(['rawScoreDiscrete'], axis=1)


df = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

/Users/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
